In [1]:
import re
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
def preprocess(text):
	lemmatizer = WordNetLemmatizer()
	text = text.lower()
	text = re.sub(r'\s+', ' ', text).strip()
	text = re.sub(r'\b(?!1[0-9]{3}\b)(?!20[0-9]{2}\b)\d+\b', '', text)
	text = re.sub(r'\s+', ' ', text).strip() 
	tokens = [lemmatizer.lemmatize(t) for t in  word_tokenize(text)]
	return ' '.join(tokens)


data = pd.read_csv('results/cleaned_topics.csv', encoding='utf-8')
dict = {number: texte for number, texte in zip(data['Number'], data['texte'])}
vectorizer = TfidfVectorizer(stop_words="english", lowercase=True,norm='l1', smooth_idf= True, sublinear_tf=True , preprocessor=preprocess)
X = vectorizer.fit_transform(data['texte'])
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(X.toarray(), columns=feature_names, index=data['Number'])
tfidf_df.to_csv('results/proba(terme sachant topic).csv', index=True, encoding='utf-8')

c:\Users\yacin\Desktop\M1\RI\M1-S2-RI\.conda\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'wa'] not in stop_words.
  warnings.warn(


In [3]:
def generate_QS1_3(topic_number, proba_terme_sachant_topic):
	serie = proba_terme_sachant_topic.loc[topic_number]
	serie = serie[serie > 0].sort_values(ascending=False)
	terms = serie.index.tolist()

	#  pivot d'abord
	matrix = np.outer(serie, serie)
	triu_indices = np.triu_indices(len(serie), k=1)
	words = serie.index.to_numpy()
	word_pairs = list(zip(words[triu_indices[0]], words[triu_indices[1]]))
	joint_probs = matrix[triu_indices]
	joint_probs += np.random.normal(0, 0.001, joint_probs.shape)
	sorted_pairs = [pair for _, pair in sorted(zip(joint_probs, word_pairs), reverse=True)]
	pivot_t1, pivot_t2 = sorted_pairs[0]

	queries = []
	used_terms = set([pivot_t1, pivot_t2]) 
	for term1 in terms:
		if term1 in used_terms:
			continue
		used_terms.add(term1)
		t3_candidates = [t for t in terms if t not in used_terms]
		if not t3_candidates:
			continue
		t3 = t3_candidates[0]
		queries.append((term1,pivot_t1, pivot_t2, t3))
		used_terms.add(t3)

	return pd.DataFrame(queries, columns=['single term', 'pivot t1', 'pivot t2', 't3'])

topic_numbers = data['Number'].unique()
for topic_number in topic_numbers:
	queries = generate_QS1_3(topic_number, tfidf_df)
	topic =  data[data['Number'] == topic_number].iloc[0]
	display(str(topic['Number']) + " : " + str(topic["texte"]))
	display(queries)


'303 : Hubble Telescope Achievements Identify positive accomplishments of the Hubble telescope since it was launched in 1991'

,single term,pivot t1,pivot t2,t3
0,1991,hubble,telescope,accomplishment
1,achievement,hubble,telescope,launched
2,positive,hubble,telescope,wa


'307 : New Hydroelectric Projects Identify hydroelectric projects proposed or under construction by country and location Detailed description of nature extent purpose problems and consequences is desirable'

,single term,pivot t1,pivot t2,t3
0,consequence,hydroelectric,project,construction
1,description,hydroelectric,project,desirable
2,detailed,hydroelectric,project,location
3,nature,hydroelectric,project,problem
4,proposed,hydroelectric,project,extent
5,new,hydroelectric,project,country
6,purpose,hydroelectric,project,identify


'310 : Radio Waves and Brain Cancer Evidence that radio waves from radio towers or car phones affect brain cancer occurrence'

...

,single term,pivot t1,pivot t2,t3
0,consumer,online,shopping,contributed
1,factor,online,shopping,growth


'648 : family leave law Identify documents that discuss details of a family leave law such as how long compensation if any for what reason allowed etc'

,single term,pivot t1,pivot t2,t3
0,law,family,leave,compensation
1,long,family,leave,allowed
2,reason,family,leave,discus
3,document,family,leave,identify


'650 : tax evasion indicted Identify individuals or corporations that have been indicted on charges of tax evasion of more than two million dollars in the US or UK'

,single term,pivot t1,pivot t2,t3
0,indicted,evasion,tax,charge
1,dollar,evasion,tax,individual
2,million,evasion,tax,uk
3,corporation,evasion,tax,identify
